# Importing Libraries

In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import librosa
from scipy.io import wavfile
!pip install audiofile
import audiofile
from scipy.signal import butter, filtfilt, lfilter, find_peaks
from scipy.fft import fft, fftshift
from IPython.display import Audio
import soundfile as sf

# Preprocessing Functions

In [2]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5*fs
    low = lowcut/nyq
    high = highcut/nyq
    b,a = butter(order, [low, high], btype='band')
    return b, a

# ------------------------------------------------------------------------------
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
  b, a = butter_bandpass(lowcut, highcut, fs, order = order)
  y = lfilter(b, a, data)
  return y

In [3]:
def Heart_Lung_Separation(pcg_signal, fs):
  pcg_filtered = butter_bandpass_filter(pcg_signal, 20, 2000, fs, order=5)
  lung_signal = butter_bandpass_filter(pcg_filtered, 100, 1000, fs, order=5)
  heart_signal = pcg_filtered - lung_signal

#   # Quality Index of heart signal:
#   heart_signal_snr = Heart_SNR(heart_signal, fs)
#   heart_quality_ind = Heart_Quality_Index(heart_signal_snr)

#   # Quality Index of lung signal:
#   lung_signal_snr = Lung_SNR(lung_signal, fs)
#   lung_quality_ind = Lung_Quality_Index(lung_signal_snr)
#   return [heart_signal, lung_signal, heart_quality_ind, lung_quality_ind]
  return [heart_signal, lung_signal]

In [4]:
# Lowpass Butterworth filter:
def butterworth_low_pass_filter(original_signal, order, cutoff, fs):
  b, a = butter(order, 2*cutoff/fs, btype = 'low')
  low_pass_filtered_signal = filtfilt(b, a, original_signal)
  return low_pass_filtered_signal

# ------------------------------------------------------------------------------
# Highpass Butterworth filter:
def butterworth_high_pass_filter(original_signal, order, cutoff, fs):
  b, a = butter(order, 2*cutoff/fs, btype = 'high')
  high_pass_filtered_signal = filtfilt(b, a, original_signal)
  return high_pass_filtered_signal

In [5]:
def heart_signal_preprocessing(heart_signal, fs):
  fc = 100  # Cutoff frequency (Hz)
  # Define the order of the filter (higher order provides better attenuation)
  order = 4
  # Define the normalized cutoff frequency (between 0 and 1, where 1 is the Nyquist frequency)
  w = fc / (fs / 2)
  b, a = butter(order, w, 'low')
  filtered_signal = filtfilt(b, a, heart_signal)
  return filtered_signal

In [6]:
def lung_signal_preprocessing(lung_signal, fs):
  # Define the filter parameters
  cutoff = 250 # Hz  100
  order = 4
  b, a = butter(order, cutoff/(fs/2), btype='highpass')
  filtered_signal = filtfilt(b, a, lung_signal)
  return filtered_signal

## Importing the pretrained models

In [7]:
heart_classifier_file_name = 'F:\HEKIDESK\My Codes\Heart_Sound_Classification_Model.h5'
heart_classifier_model = load_model(heart_classifier_file_name)
lung_classifier_file_name = 'F:\HEKIDESK\My Codes\Lung_Sound_Classification_Model.h5'
lung_classifier_model = load_model(lung_classifier_file_name)

## Prediction

In [8]:
def Heart_Class_Prediction (signal, fs):
    classes = ["murmur","normal"]
    signal = np.array(np.float32(signal))
    mfccs = np.mean(librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=52, n_fft=512, hop_length=2048).T,axis=0)
#     feature = np.array(mfccs).reshape([-1,1])
    preds = heart_classifier_model.predict(mfccs.reshape(1,52,1))
    confidence = np.amax(preds)
    prediction = classes[np.argmax(preds)]
    return prediction, confidence 

## -------------------------------------------------------------------------------------------------------------
def Lung_Class_Prediction(signal, fs):
    classes = ["normal","crackle","wheezes"]
    signal = np.array(np.float32(signal))
    mfccs = np.mean(librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=52, n_fft=512,hop_length=2048).T,axis=0)
#     feature = np.array(mfccs).reshape([-1,1])
    preds = lung_classifier_model.predict(mfccs.reshape(1,52,1))
    print(preds)
    
    confidence = np.amax(preds)
    prediction = classes[np.argmax(preds)]
    return prediction, confidence 

## Testing the model

In [9]:
with open('pcg_15.txt') as fid:
    data = fid.readlines()

new_data = list(map(lambda s: s.strip(), data))
new_data.remove('[')
new_data.remove(']')

for i in range(len(new_data)):
    new_data[i] = new_data[i].replace(',', '')

sound_signal = list(map(int, new_data))
fs = int(np.floor(len(sound_signal)/20))
heart_signal, lung_signal = Heart_Lung_Separation(sound_signal, fs)
heart_signal = heart_signal_preprocessing(heart_signal, fs)
lung_signal = lung_signal_preprocessing(lung_signal, fs)

heart_prediction, heart_confidence = Heart_Class_Prediction (heart_signal, fs)
print('Heart sound class predicted is: ', heart_prediction)
print('Heart sound class is predicted with ', heart_confidence, '% confidence.')

1/1 [==============================] - 2s 2s/step
Heart sound class predicted is:  murmur
Heart sound class is predicted with  0.9999018 % confidence.


In [10]:
lung_prediction, lung_confidence = Lung_Class_Prediction (lung_signal, fs)
print('Lung sound class predicted is: ', lung_prediction)
print('Lung sound class is predicted with ', lung_confidence, '% confidence.')

ValueError: in user code:

    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Asus\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 o